#### LeetCode - SQL - #601

Refer [601. Human Traffic of Stadium](https://leetcode.com/problems/human-traffic-of-stadium/description/)

Write a solution to display the records with three or more rows with  **consecutive**  `id`'s, and the number of people is greater than or equal to 100 for each.

Return the result table ordered by  `visit_date`  in  **ascending order**.

In [ ]:
data_stadium = [[1, '2017-01-01', 10], [2, '2017-01-02', 109], [3, '2017-01-03', 150], [4, '2017-01-04', 99], [5, '2017-01-05', 145], [6, '2017-01-06', 1455], [7, '2017-01-07', 199], [8, '2017-01-09', 188]]
columns_stadium = ['id', 'visit_date', 'people']
schema_stadium = {'id':'Int64', 'visit_date':'datetime64[ns]', 'people':'Int64'}

In [ ]:
import pandas

pandas_df_stadium = pandas.DataFrame(data=data_stadium, columns=columns_stadium).astype(schema_stadium)

In [ ]:
from pyspark.sql import SparkSession

spark_context = SparkSession.builder.appName('LeetCode SQL').getOrCreate()

# Spark DataFrame using Pandas DataFrame
df_stadium = spark_context.createDataFrame(pandas_df_stadium)
df_stadium.printSchema()
df_stadium.show(5)

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F

window_spec = Window.orderBy(df_stadium.id.asc())

df_stadium_summary \
    = df_stadium \
        .filter(df_stadium.people >= 100) \
        .select("*", (df_stadium.id - (F.row_number().over(window_spec))).alias("record_pos"))
df_stadium_summary.show()

In [ ]:
df_stadium_summary_record_pos \
    = df_stadium_summary \
        .groupBy(df_stadium_summary.record_pos) \
        .agg(F.count("*").alias("record_pos_count"))
df_stadium_summary_record_pos.show()

In [ ]:
df_stadium_summary_record_pos_values \
    = df_stadium_summary_record_pos \
        .filter(df_stadium_summary_record_pos.record_pos_count >= 3) \
        .select(df_stadium_summary_record_pos.record_pos)

list_stadium_summary_record_pos_values = list(df_stadium_summary_record_pos_values.toPandas()["record_pos"])

In [ ]:
df_stadium_summary \
    .filter(df_stadium_summary.record_pos.isin(list_stadium_summary_record_pos_values)) \
    .select(df_stadium_summary.id, df_stadium_summary.visit_date, df_stadium_summary.people) \
    .show()